In [2]:
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import column, row, widgetbox, gridplot
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider
from bokeh.models import mappers, CustomJS, ColumnDataSource, ColorBar, tickers
import numpy as np

Loading BokehJS ...

In [3]:
def delF(F, dx):
    return (-4*F + 
            np.roll(F, 1, axis=0) + 
            np.roll(F, -1, axis=0) + 
            np.roll(F, 1, axis=1) + 
            np.roll(F, -1, axis=1)) / (dx*dx)

In [7]:
# set up constants
# number of grid points
Nx = 100
Ny = 100
# grid size
dx = 0.1
# time
t0 = 0.0
tf = 5.0
dt = 0.0002
# calc number of steps
nSteps = (tf - t0)/dt
# set number of steps b/w plots
numPlots = 3
nPlot = int(nSteps / numPlots)
# model params
W = 1.0
eps = 0.1
M = 1.0
C0avg = 0.5
C0amp = 0.1

C = (2 * np.random.rand(Nx, Ny) - 1) * C0amp + C0avg
# this array holds the individual frames that will be displayed
myC = np.array([C, C, C])

t = t0

my_map = mappers.LinearColorMapper(palette="Viridis256", low=0.0, high=1.0)

p_list = list()
p_list.append(figure(title="t_0", x_range=(0,Nx), y_range=(0, Ny)))
image_C = np.ones(shape=(Nx, Ny, 4))
p_list[0].image(image=[myC[0]], x=0, y=0, dw=Nx, dh=Ny, color_mapper=my_map)
axnum = 0
myIter = 0
myC[myIter] = C
while t < tf:
    for i in range(0, nPlot):
        lapC = delF(C, dx)
        dfdc = (W / 2.0) * np.multiply(np.multiply(C, 1.0 - C), 1.0 - 2.0 * C)
        mu = dfdc - (eps * eps) * lapC
        lapMu = delF(mu, dx)
        dcdt = M * lapMu
        C += dcdt * dt
        t += dt
    axnum += 1
    if axnum > 2:
        break
    myIter += 1
    myC[myIter] = C
    p_list.append(figure(title="t={:.4f}".format(t), x_range=(0,Nx), y_range=(0, Ny)))
    p_list[-1].image(image=[myC[myIter]], x=0, y=0, dw=Nx, dh=Ny, color_mapper=my_map)
grid=gridplot(p_list, ncols=numPlots, plot_width=300, plot_height=300)
show(grid)